In [1]:
import xarray as xr
import numpy as np
import dask
import matplotlib.pyplot as plt
from utils import geo
import pandas as pd
import regionmask
from dask.diagnostics import ProgressBar
import glob
import os

In [2]:
dataset = 'globcolour-cmems'
frequency = 'day'
variable = 'CHL1'
yearstart = 1999
yearend = 2022
monthorder = 'JULtoJUN'
averaging = 'wmean-latlon'

In [3]:
if dataset == 'globcolour-hermes':
    rootdir = '/projects/SOCCOM/datasets/GlobColour/'
    local = 'HERMES/merged/'+frequency+'/*/*/*/'
    mode = 'AV'
    filename = '*'+mode+'*'+variable+'*.nc'
    
    def preprocess(ds):
        time = pd.to_datetime(ds.attrs['start_time'])
        if 'CHL1_error' in ds.data_vars:
            ds = ds.drop(['CHL1_error'])
        ds = ds.sel({'lat':slice(-30,-65)}).expand_dims({'time':[time]}).rename({'CHL1_mean':variable})
        return ds
    
elif dataset == 'globcolour-cmems':
    rootdir = '/projects/SOCCOM/datasets/GlobColour/'
    local = 'CMEMS/OCEANCOLOUR_GLO_BGC_L4_MY_009_104-TDS/SouthernOcean_30Sto65S/'+frequency+'/bymonth/'
    filename = 'cmems_obs-oc_glo_bgc-plankton_my_l4-gapfree-multi-4km_P1D_*.nc'
    
    def preprocess(ds):
        time = pd.to_datetime(ds.attrs['start_time'])
        # This is a hack because of issues with data download
        ds = ds.where(ds['time.month']==ds['time.month'][0],drop=True).rename({'CHL':variable})
        return ds.copy()

print('Opening.')
paths = glob.glob(rootdir+local+filename)
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds = xr.open_mfdataset(paths,preprocess=preprocess,use_cftime=False)
ds['time'] = pd.to_datetime(ds['time'].values)
# Get area
ds,xgrid = geo.get_xgcm_horizontal(ds,axes_dims_dict={'X':'lon','Y':'lat'},periodic='X')
ds['area'] = ds['dxC']*ds['dyC']

Opening.


In [4]:
ds

<xarray.Dataset>
Dimensions:          (time: 9202, lat: 841, lon: 8640, lon_left: 8640,
                      lat_left: 841)
Coordinates:
  * lon              (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time             (time) datetime64[ns] 1997-09-04 1997-09-06 ... 2023-01-04
  * lat              (lat) float32 -29.98 -30.02 -30.06 ... -64.9 -64.94 -64.98
  * lon_left         (lon_left) float32 -180.0 -180.0 -179.9 ... 179.9 180.0
  * lat_left         (lat_left) float32 -29.96 -30.0 -30.04 ... -64.92 -64.96
Data variables:
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(19, 841, 8640), meta=np.ndarray>
    CHL1             (time, lat, lon) float32 dask.array<chunksize=(19, 841, 8640), meta=np.ndarray>
    flags            (time, lat, lon) float64 dask.array<chunksize=(19, 841, 8640), meta=np.ndarray>
    dxG              (lon_left, lat) float32 4.005e+03 4.004e+03 ... 1.956e+03
    dyG              (lon, lat_left) float32 nan -4.625e+03 ... -4.625e+03
    dxC              (lon, lat) float32 4.005e+03 4.004e+03 ... 1.956e+03
    dyC              (lon, lat) float32 -4.625e+03 -4.625e+03 ... -4.626e+03 nan
    rC               (lon, lat) float32 -1.852e+07 -1.852e+07 ... -9.06e+06 nan
    area             (lon, lat) float32 -1.852e+07 -1.852e+07 ... -9.06e+06 nan
Attributes: (12/92)
    FROM_ORIGINAL_FILE__lon_step:                     0.041666668
    keywords:                                         EARTH SCIENCE > OCEANS ...
    cmems_product_id:                                 OCEANCOLOUR_GLO_BGC_L4_...
    FROM_ORIGINAL_FILE__lat_step:                     0.041666668
    grid_mapping:                                     Equirectangular
    title:                                            cmems_obs-oc_glo_bgc-pl...
    ...                                               ...
    geospatial_vertical_positive:                     up
    date_created:                                     2023-01-11T06:44:09Z
    nb_valid_bins:                                    19933570
    pct_valid_bins:                                   53.40569808813443
    parameter:                                        Chlorophyll-a concentra...
    _CoordSysBuilder:                                 ucar.nc2.dataset.conv.C...

In [5]:
ds = ds.chunk({'time':5})
ds

<xarray.Dataset>
Dimensions:          (time: 9202, lat: 841, lon: 8640, lon_left: 8640,
                      lat_left: 841)
Coordinates:
  * lon              (lon) float32 -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * time             (time) datetime64[ns] 1997-09-04 1997-09-06 ... 2023-01-04
  * lat              (lat) float32 -29.98 -30.02 -30.06 ... -64.9 -64.94 -64.98
  * lon_left         (lon_left) float32 -180.0 -180.0 -179.9 ... 179.9 180.0
  * lat_left         (lat_left) float32 -29.96 -30.0 -30.04 ... -64.92 -64.96
Data variables:
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(5, 841, 8640), meta=np.ndarray>
    CHL1             (time, lat, lon) float32 dask.array<chunksize=(5, 841, 8640), meta=np.ndarray>
    flags            (time, lat, lon) float64 dask.array<chunksize=(5, 841, 8640), meta=np.ndarray>
    dxG              (lon_left, lat) float32 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    dyG              (lon, lat_left) float32 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    dxC              (lon, lat) float32 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    dyC              (lon, lat) float32 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    rC               (lon, lat) float32 dask.array<chunksize=(8640, 841), meta=np.ndarray>
    area             (lon, lat) float32 dask.array<chunksize=(8640, 841), meta=np.ndarray>
Attributes: (12/92)
    FROM_ORIGINAL_FILE__lon_step:                     0.041666668
    keywords:                                         EARTH SCIENCE > OCEANS ...
    cmems_product_id:                                 OCEANCOLOUR_GLO_BGC_L4_...
    FROM_ORIGINAL_FILE__lat_step:                     0.041666668
    grid_mapping:                                     Equirectangular
    title:                                            cmems_obs-oc_glo_bgc-pl...
    ...                                               ...
    geospatial_vertical_positive:                     up
    date_created:                                     2023-01-11T06:44:09Z
    nb_valid_bins:                                    19933570
    pct_valid_bins:                                   53.40569808813443
    parameter:                                        Chlorophyll-a concentra...
    _CoordSysBuilder:                                 ucar.nc2.dataset.conv.C...

In [6]:
# Apply a landmask
mask = regionmask.defined_regions.natural_earth.land_110.mask(ds['lon'], ds['lat'])
mask = mask.where(np.isfinite(mask),1)
mask = mask.where(mask==1,np.nan)
# mask.plot()
ds[variable] = ds[variable].fillna(0)*mask

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/regionmask/defined_regions/_natural_earth.py:471: FutureWarning: ``regionmask.defined_regions.natural_earth`` is deprecated. Please use ``regionmask.defined_regions.natural_earth_v4_1_0`` or ``regionmask.defined_regions.natural_earth_v5_0_0`` instead.
  warnings.warn(


In [7]:
# Extract spatial average for each year
years = np.arange(yearstart,yearend+1)
days = np.arange(0,365)
# Ordering such that years run from July to June
ds_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    if monthorder=='JULtoJUN':
        start = str(year-1)+'-07-01'
        end = str(year)+'-06-30'
        nleapday = 243 # for removing leap day (243 days after July 1)
    elif monthorder=='JANtoDEC':
        start = str(year-1)+'-01-01'
        end = str(year)+'-12-31'
        nleapday = 59 # for removing leap day (59 days after Jan 1)
    
    # Weighted spatial mean
    x = (ds[variable].sel({'time':slice(start,end)})).weighted(ds['area'].fillna(0)).mean(['lat','lon'])
    
    # Interpolate to daily
    # (day frquency data has some missing days, so interpolate that too)
    alltimes = pd.date_range(start,end)
    xi = x.interp({'time':alltimes},kwargs={'fill_value':np.nan})
        
    # Remove leap-year day for simplicity
    if len(xi)==366:
        xi = xi[np.arange(len(xi))!=nleapday]
        
    # Put into dataset
    with ProgressBar():
        xi = xi.compute()
    ds_year.loc[{'year':year}]=xi.values

1999
[########################################] | 100% Completed | 148.64 s
2000
[########################################] | 100% Completed | 121.99 s
2001
[########################################] | 100% Completed | 99.14 s
2002
[########################################] | 100% Completed | 101.29 s
2003
[########################################] | 100% Completed | 106.07 s
2004
[########################################] | 100% Completed | 105.88 s
2005
[########################################] | 100% Completed | 99.16 s
2006
[########################################] | 100% Completed | 104.47 s
2007
[########################################] | 100% Completed | 104.34 s
2008
[########################################] | 100% Completed | 111.94 s
2009
[########################################] | 100% Completed | 116.19 s
2010
[########################################] | 100% Completed | 106.98 s
2011
[########################################] | 100% Completed | 108.48 s
2012
[########

In [8]:
outdir = '../../data/'
yearstr = str(yearstart)+'-'+str(yearend)
outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,'nc'])

In [9]:
print('Saving to '+outpath)
if os.path.isfile(outpath):
    os.remove(outpath)
ds_year.to_netcdf(outpath)

Saving to ../../data/globcolour-cmems.day.wmean-latlon.JULtoJUN.1999-2022.nc


### By basin

In [7]:
atlind = 20
indwpac = 170
wpacepac = -130
epacatl = -70

cond_atlantic = (ds['lon']>=epacatl) & (ds['lon']<atlind)
cond_indian = (ds['lon']>=atlind) & (ds['lon']<indwpac)
cond_wpacific = (ds['lon']>=indwpac) | (ds['lon']<wpacepac)
cond_epacific = (ds['lon']>=wpacepac) & (ds['lon']<epacatl)

atlantic = ds[variable].where(cond_atlantic,drop=True)
atlantic_area = ds['area'].where(cond_atlantic,drop=True)
indian = ds[variable].where(cond_indian,drop=True)
indian_area = ds['area'].where(cond_indian,drop=True)
wpacific = ds[variable].where(cond_wpacific,drop=True)
wpacific_area = ds['area'].where(cond_wpacific,drop=True)
epacific = ds[variable].where(cond_epacific,drop=True)
epacific_area = ds['area'].where(cond_epacific,drop=True)

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/graemem/.conda/envs/core/lib/python3.10/site-packages/xarray/core/indexing.py:1228: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/home/

In [8]:
years = np.arange(yearstart,yearend+1)
days = np.arange(0,365)
# Extract spatial average for each year
atlantic_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
indian_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
wpacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
epacific_year = xr.DataArray(dims=['day','year'],coords={'year':years,'day':days},name=variable)
for i,year in enumerate(years):
    print(year)
    start = str(year-1)+'-07-01'
    end = str(year)+'-06-30'
    alltimes = pd.date_range(start,end)
    
    # Atlantic
    print('atlantic')
    x = (atlantic.sel({'time':slice(start,end)})).weighted(atlantic_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        atlantic_year.loc[{'year':year}]=x.values
        
    # Indian
    print('indian')
    x = (indian.sel({'time':slice(start,end)})).weighted(indian_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        indian_year.loc[{'year':year}]=x.values
        
    # West Pacific
    print('wpacific')
    x = (wpacific.sel({'time':slice(start,end)})).weighted(wpacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        wpacific_year.loc[{'year':year}]=x.values
        
    # East Pacific
    print('epacific')
    x = (epacific.sel({'time':slice(start,end)})).weighted(epacific_area.fillna(0)).mean(['lat','lon'])
    x = x.interp({'time':alltimes})
    if len(x)==366:
        # Just take out the leap year day
        # (Always 243 days after July 1)
        x = x[np.arange(len(x))!=243]
    # Put into dataset
    with ProgressBar():
        epacific_year.loc[{'year':year}]=x.values

1999
atlantic
[########################################] | 100% Completed | 87.44 s
indian
[########################################] | 100% Completed | 85.80 s
wpacific
[########################################] | 100% Completed | 67.67 s
epacific
[########################################] | 100% Completed | 67.40 s
2000
atlantic
[########################################] | 100% Completed | 89.70 s
indian
[########################################] | 100% Completed | 87.75 s
wpacific
[########################################] | 100% Completed | 69.21 s
epacific
[########################################] | 100% Completed | 68.91 s
2001
atlantic
[########################################] | 100% Completed | 78.38 s
indian
[########################################] | 100% Completed | 83.53 s
wpacific
[########################################] | 100% Completed | 66.37 s
epacific
[########################################] | 100% Completed | 66.63 s
2002
atlantic
[############################

In [10]:
outdir = '../../data/'
yearstr = str(yearstart)+'-'+str(yearend)

for basin in ['atlantic','indian','wpacific','epacific']:
    outpath = outdir+'.'.join([dataset,frequency,averaging,monthorder,yearstr,basin,'nc'])
    if basin=='atlantic':
        atlantic_year.to_netcdf(outpath)
    elif basin=='indian':
        indian_year.to_netcdf(outpath)
    elif basin=='wpacific':
        wpacific_year.to_netcdf(outpath)
    elif basin=='epacific':
        epacific_year.to_netcdf(outpath)